# <center>Cryptocurrency arbitrage finding application</center> 
##### By: Bartłomiej Kowalczuk & Michał Thor

Our application uses selenium to scrape data for cryptocurrencies from <u>https://coinmarketcap.com/exchanges/bitmax/</u> and process it with our algorithm which looks for arbitrage options (riskless profit) by exchanging currencies within BitMax market. 

Additionally this application has a feature to download historical prices and volume of 4 most traded cryptocurrencies also using selenium scraper.

Usage of the application is presented in steps below.

### Firstly, we need to install all the required packages. For that purpose we use pip install ...

In [ ]:
!pip install -r requirements.txt

### Secondly, we have to download appropriate version of chromedriver.exe file from <u>http://chromedriver.chromium.org/</u> and save it directly to our *C://* drive. (There is option to change the Path when initializing class at the end of *app.py* file if we want to store your chromedriver.exe elsewhere).

### Then, we can start the application. (*All the required packages are imported within the app.py*)

In [5]:
!python app.py

### After running application we can see the starting window with <u>Find triangular arbitrage</u> button on the top and <u>Options</u> tab in menu bar. 

![Startingscreen](screenshots/01.JPG)

Upon clicking the button on the top we start the selenium scraper (*all codes are provided in .py files attached in the e-mail and also at the end of this notebook*) along with chromedriver.exe and afterwards arbitrage finding function (*codes provided as for scraper*) which uses all the necessary scrapped information. After the scrapping, chromedriver.exe shuts down so it is appearing on screen just for a brief moment while it is working.

After the scrapping has been finished, in the application window we can see information about possible arbitrage strategies using triangular arbitrage. Strategies are calculated starting up with up to 4 of the most traded cryptocurrencies (*in the first line of each strategy there is information about how much currency we are using, e.g. 100 BTC(bitcoin) and its' percantage share of overall daily volume - so how liquid is that cryptocurrency*) but if there is no possibility of arbitrage, the strategy for that currency will not be displayed.

In following lines we can see number of operation, how it was conducted and what was the profit in the original currency. After each operation, amount of the original currency is increased by the profit from last operation. Operations continue till there is no more room to gain profit without risk.

After all of the operations are done for particular currency we see how much USD (*US dollar*) we obtained (*using scraped CURRENCY/USD rate*).

![Strategies](screenshots/02.JPG)

### Under the text box containing strategies we can see the <u>Clear</u> button which lets us clear the output and perform the arbitrage calculation again.

### Additionally we can scrape historic prices and volume from the last 30 days for those previously mentioned 4 currencies (*all the prices are denominated in USD for better readability*).

### To do this we have to click on the <u>Options</u> button in the left upper corner and then on the <u>Browse currency history</u> button. 

Chromedriver will run once again and shut itself after scrapping all the needed information.

In the application window the plot for prices should be visible. Below the plot we have <u>Show rates/volumes graph</u> which will swith between two views. Underneath again we have the <u>Clear</u> button which clears output and lets us perform other actions.

![Rates](screenshots/03.JPG) ![Volumes](screenshots/04.JPG)

##  <center>----------------------------------------------------------------------------------------------------------------</center>
## Although our application works in a moderatly quick time (but not quick enough, yet) its' usage is limited. It is useful under the assumption that all the pairs of the cryptocurrencies can be liquidated almost immediatly and market such as BitMax does not prevent usage of trading bots.
## <center>----------------------------------------------------------------------------------------------------------------</center>

# <center>SOURCE CODE</center>
# <center>--------------------------------------------------------</center>

### Rates scraper and arbitrage calculator

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

class Arbitrage:

    def __init__(self, file_path = 'C:\\chromedriver.exe'):
        self.file_path = file_path
        self.getRates()


    def startDriver(self):
        option = webdriver.ChromeOptions()
        option.add_argument(" — incognito")
        browser = webdriver.Chrome(executable_path=self.file_path)
        browser.get("https://coinmarketcap.com/exchanges/bitmax/")
        self.browser = browser


    def getRates(self):
        exchange_lists = []
        self.startDriver()
        curr_tab = self.browser.find_elements_by_xpath('//table[@id="exchange-markets"]/tbody/tr')
        for tr in curr_tab:
            single_exchange = []
            td = tr.find_elements_by_tag_name("td")
            for i in range(1,6):
                if len(td[i].find_elements_by_class_name("price")) == 0:
                    single_exchange.append(td[i].get_attribute("data-sort"))
                else:
                    single_exchange.append(td[i].find_element_by_class_name("price").get_attribute("data-native"))
                    single_exchange.append(td[i].get_attribute("data-sort"))
            exchange_lists.append(single_exchange)
        df = pd.DataFrame(exchange_lists, columns = ["Currency", "Pair", "Volume", "Rate", "PriceUSD", "VolumePerc"])
        df.Rate = df.Rate.astype(float)
        df[['Numerator','Denumerator']] = df["Pair"].str.split("/",expand=True,)
        self.df = df


    def prepareData(self, df, initialCoin = 'BTC'):
        # INVERSE RATES CREATION
        # append copy of dataframe to create inverse rates
        df2 = df.copy()
        df2.Rate = list(map(lambda x: float(1/x), df2.Rate))
        df2 = df2.rename(columns={'Numerator':'Denumerator','Denumerator':'Numerator'})

        df = (df
                .append(df2, sort=False)
                .reset_index()
                .drop(columns=['index'])
             )

        # POSSIBLE PAIRS CREATION
        pairList = []
        # for every denumerator that has BTC as numerator
        for i in list(df.Denumerator[df['Numerator'] == initialCoin]):
            # for every denumerator that has i as numerator
            for j in list(set((df.Denumerator[(df['Numerator'] == i) & (df['Numerator'] != initialCoin) & (df['Denumerator'] != initialCoin)]))):
                if j in list(df.Denumerator[df['Numerator'] == initialCoin]):
                    pairList.append([i,j])

        return df, pairList


    def triangularArbitrage(self, df, inputSet, firstTransaction, secondTransaction, fee):
        # first transaction
        output1 = inputSet[0]/df.Rate.loc[(df['Numerator'] == inputSet[1]) & (df['Denumerator'] == firstTransaction)].values[0]
        output1 = output1 - fee*output1

        # second transaction
        output2 = output1/df.Rate.loc[(df['Numerator'] == firstTransaction) & (df['Denumerator'] == secondTransaction)].values[0]
        output2 = output2 - fee*output2

        # third transaction
        output3 = output2/df.Rate.loc[(df['Numerator'] == secondTransaction) & (df['Denumerator'] == inputSet[1])].values[0]
        output2 = output2 - fee*output2

        return float(output3 - inputSet[0])


    def getUSDPrices(self):
        prices = (self.df[self.df.Currency.isin(['Bitcoin', 'Paxos Standard Token', 'USD Coin', 'Ethereum'])]
               .groupby('Numerator')['PriceUSD']
               .max()
               .to_dict())
        self.pricesUSD = prices


    def findArbitrage(self, quantity = 1):
        baseCoins = ['BTC', 'PAX', 'ETH', 'USDC']
        self.getUSDPrices()
        transactionList = []
        for coin in baseCoins:
            transactionList.append(str(f" Arbitrage using {quantity} {coin} -- {round(float(max(self.df.VolumePerc[self.df.Numerator == coin])), 2)} % of market daily volume"))
            df = self.df.copy()
            df, pairList = self.prepareData(df = df, initialCoin = coin)
            suma = 0
            n = 1
            quantity2 = quantity
            for i in pairList:
                add = self.triangularArbitrage(df = df, inputSet = (quantity2, coin),
                                               firstTransaction = i[0], secondTransaction = i[1], fee = 0.0004)
                if add > 0:
                    suma += add
                    quantity2 += add
                    transactionList.append(str(f'        {n} operation: {coin} -> {i[0]} -> {i[1]} -> {coin}: {round(add, 5)} {coin}'))
                    n += 1
            if round(suma * float(self.pricesUSD[coin]), 2) > 0:
                transactionList.append(str(""))
                transactionList.append(str(f' Obtained: {round(suma * float(self.pricesUSD[coin]), 2)} USD by {n} triangular transactions using cummulated {coin} asset'))
                transactionList.append(str(f"Percentage profit achived: {round(((suma)/100)*100,2)}%"))
                transactionList.append(str(""))
                transactionList.append(str('--------------------------------------'))
                transactionList.append(str(""))
            else:
                transactionList.append(str("No arbitrage possibilities for this cryptocurrency at this moment."))
        time.sleep(5)
        self.browser.quit()
        return transactionList


# Instantiate an object of type Arbitrage (with path to chromedriver.exe on your machine)
# kA = Arbitrage('C:\\chromedriver.exe')
# look for arbitrage opportunities on BitMax exchange (set the desired, initial quantity of currencies)
# kA.findArbitrage(quantity = 100)
# kA.getRates()
# print(max(kA.df.VolumePerc[kA.df.Numerator == 'USDC']))


### Historic information (*for plots*) scraper

In [ ]:
import pandas as pd
from selenium import webdriver
import itertools
import time
import re
import matplotlib.pyplot as plt

class CryptoHistory():
    def __init__(self, file_path = 'C:\\chromedriver.exe'):
        self.file_path = file_path
        self.startDriver()
        self.getUrls()

    def striplist(self, list):
        return([x.replace(' ', '') for x in list])

    def startDriver(self):
        option = webdriver.ChromeOptions()
        option.add_argument(" — incognito")
        browser = webdriver.Chrome(executable_path=self.file_path)
        browser.get("https://coinmarketcap.com/exchanges/bitmax/")
        self.browser = browser

    def getUrls(self):
        urls = []
        infos = self.browser.find_elements_by_class_name('margin-left--lv1')
        urls = [(url.get_attribute("href") + "historical-data") for url in infos]
        mainUrls = []
        for item in urls:
            if re.search("bitcoin/|paxos|ethereum/|usd-coin", item):
                mainUrls.append(item)
        mainUrls = set(mainUrls)
        self.urls = mainUrls
        currencyNames = []
        for item in mainUrls:
            currencyNames.append((re.findall('currencies/(\w+-?\w+-?\w+)/historical-data', item)[0]))
        self.currencyNames = currencyNames

    def currHistory(self, url):
        self.browser.get(url)
#         time.sleep(5)
        exchange_lists = []
        curr_tab = self.browser.find_elements_by_xpath('//table[@class="table"]//tr')
        exchange_lists = [(td.text for td in tr.find_elements_by_xpath(".//*[self::td or self::th]")) for tr in curr_tab]
        df = pd.DataFrame(exchange_lists)
        df.columns = df.iloc[0]
        df = df.reindex(df.index.drop(0))
        df.sort_index(inplace = True, ascending = False)

        high_price = self.striplist(list(df.iloc[:,2].transpose()))
        volume = self.striplist(list(df.iloc[:,5].transpose()))
        return high_price, volume

    def historyToDataFrames(self):
        prices = []
        volumes = []

        for url in self.urls:
            price, volume = self.currHistory(url)
            prices.append(price)
            volumes.append(volume)

        df_prices = pd.DataFrame(prices).transpose()
        ratesColnames = [s + ' rate' for s in self.currencyNames]
        df_prices.columns = ratesColnames
        df_volumes = pd.DataFrame(volumes).transpose()
        volumesColnames = [s + ' volume' for s in self.currencyNames]
        df_volumes.columns = volumesColnames

        df_prices = df_prices.apply(lambda x: x.str.replace(',','.').astype(float))
        self.df_prices = df_prices
        df_volumes = df_volumes.apply(lambda x: x.str.replace(',','.').astype(float))
        self.df_volumes = df_volumes
        time.sleep(5)
        self.browser.quit()

    def plotHistoryData(self):
        self.df_prices.plot(title = 'Currency rates')
        plt.show()
        self.df_volumes.plot(title = 'Currency volumes')
        plt.show()

# # Instantiate an object of type CryptoHistory (with path to chromedriver.exe on your machine)
# testObject = CryptoHistory(file_path = 'C:\\chromedriver.exe')
# get historical data for initital coins
# testObject.historyToDataFrames()
# plot that data
# testObject.plotHistoryData()


### Main application

In [ ]:
from tkinter import *
import pandas as pd
from PIL import Image, ImageTk
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

import itertools
import time
import re
import matplotlib.pyplot as plt

from arbitrage import Arbitrage
from historyScraper import CryptoHistory

class CryptoArbitrage(Frame):

    def __init__(self, master=None, file_path="C:\\chromedriver.exe"):
        Frame.__init__(self, master)
        self.master = master
        self.init_window()
        self.file_path = file_path

    def init_window(self):

        self.master.title("Cryptocurrency arbitrage")
        self.pack(fill=BOTH, expand=1)

        load = Image.open("trading.png")
        render = ImageTk.PhotoImage(load)
        img = Label(self, image=render)
        img.image = render
        img.place(x=0, y=0)

        # creating a menu instance
        menu = Menu(self.master)
        self.master.config(menu=menu)
        edit = Menu(menu)
        edit.add_command(label="Browse currency history", command=self.showHistory)
        menu.add_cascade(label="Options", menu=edit)

        button = Button(self, text="Find triangular arbitrage", command=self.showText)
        button.place(relx = 0.1, rely = 0.3, anchor="center")
        button.pack()

    def showText(self):
        load = Image.open("trading.png")
        render = ImageTk.PhotoImage(load)
        img = Label(self, image=render)
        img.image = render
        img.place(x=0, y=0)

        kA = Arbitrage(self.file_path)
        transactionList = kA.findArbitrage(quantity = 100)

        scrollbar = Scrollbar(self, orient=VERTICAL)
        listbox = Listbox(self, width=110, height=35, yscrollcommand=scrollbar.set)
        scrollbar.config(command=listbox.yview)
        scrollbar.pack(side=RIGHT, fill=Y)
        for listId, transaction in enumerate(transactionList):
            listbox.insert(listId, transaction)
        listbox.place(relx = 0.1, rely = 0.3, anchor="center")
        listbox.pack()

        button = Button(self, text="Clear", command=self.clearOutput)
        button.place(relx = 0.1, rely = 0.3, anchor="center")
        button.pack()


    def showHistory(self):
        load = Image.open("trading.png")
        render = ImageTk.PhotoImage(load)
        img = Label(self, image=render)
        img.image = render
        img.place(x=0, y=0)

        testObject = CryptoHistory(file_path = self.file_path)
        testObject.historyToDataFrames()

        self.df_prices = testObject.df_prices
        self.df_volumes = testObject.df_volumes

        figure1 = plt.Figure(figsize=(6,5), dpi=100)
        ax1 = figure1.add_subplot(111)
        ax1.set(
            xlabel = "Time (last 30 days)",
            ylabel = "Price (in USD)",
            xticklabels= []
            )
        bar1 = FigureCanvasTkAgg(figure1, self)
        bar1.get_tk_widget().pack()
        self.df_prices.plot(title='Currency rates', ax=ax1)

        button = Button(self, text="Show volumes graph", command=self.showVolumeGraph)
        button.place(relx = 0.1, rely = 0.3, anchor="center")
        button.pack()

        button = Button(self, text="Clear", command=self.clearOutput)
        button.place(relx = 0.1, rely = 0.3, anchor="center")
        button.pack()


    def showVolumeGraph(self):
        for widget in self.winfo_children():
            widget.destroy()

        load = Image.open("trading.png")
        render = ImageTk.PhotoImage(load)
        img = Label(self, image=render)
        img.image = render
        img.place(x=0, y=0)

        figure1 = plt.Figure(figsize=(6,5), dpi=100)
        ax1 = figure1.add_subplot(111)
        ax1.set(
            xlabel = "Time (last 30 days)",
            ylabel = "Number of operations (in bln)",
            xticklabels= []
            )
        bar1 = FigureCanvasTkAgg(figure1, self)
        bar1.get_tk_widget().pack()
        self.df_volumes.plot(title='Currency volumes', ax=ax1)

        button = Button(self, text="Show rates graph", command=self.showRatesGraph)
        button.place(relx = 0.1, rely = 0.3, anchor="center")
        button.pack()

        button = Button(self, text="Clear", command=self.clearOutput)
        button.place(relx = 0.1, rely = 0.3, anchor="center")
        button.pack()


    def showRatesGraph(self):
        for widget in self.winfo_children():
            widget.destroy()

        load = Image.open("trading.png")
        render = ImageTk.PhotoImage(load)
        img = Label(self, image=render)
        img.image = render
        img.place(x=0, y=0)

        figure1 = plt.Figure(figsize=(6,5), dpi=100)
        ax1 = figure1.add_subplot(111)
        ax1.set(
            xlabel = "Time (last 30 days)",
            ylabel = "Price (in USD)",
            xticklabels= []
            )
        bar1 = FigureCanvasTkAgg(figure1, self)
        bar1.get_tk_widget().pack()
        self.df_prices.plot(title='Currency rates', ax=ax1)

        button = Button(self, text="Show volumes graph", command=self.showVolumeGraph)
        button.place(relx = 0.1, rely = 0.3, anchor="center")
        button.pack()

        button = Button(self, text="Clear", command=self.clearOutput)
        button.place(relx = 0.1, rely = 0.3, anchor="center")
        button.pack()

    def clearOutput(self):
        for widget in self.winfo_children():
            widget.destroy()

        load = Image.open("trading.png")
        render = ImageTk.PhotoImage(load)
        img = Label(self, image=render)
        img.image = render
        img.place(x=0, y=0)

        button = Button(self, text="Find triangular arbitrage", command=self.showText)
        button.place(relx = 0.1, rely = 0.3, anchor="center")
        button.pack()


# Commented initialization of application so it does not run upon executing this cell        

# top = Tk()
# top.geometry("1000x700")
# app = CryptoArbitrage(top, "C:\\chromedriver.exe")
# top.mainloop()
